# Data transformation, Staging area load, multidimensional data model for data warehouse. 

Given northwind databased diagram as:

<img src="D:\Project_pipeline\data_cube_diagram.png" alt="alt text" width="width" height="height">

We need to create data cube like this:


<img src="D:\Project_pipeline\nortwind_diagram.png" alt="alt text" width="700" height="700">

## customer dimension table.

1. Read customer table from `customer.csv` file.

2. Create timstamp column with column name `ingestion_timestamp` and name table as `stg_customer`, and load data into `stg_northwind` database. 

3. Transform data as follows:
    - Sort by `id` and get unique `id`.
    - Rename `id` columns to `customer_id`.
    - Delete an `ingestion_timestamp` column and add timstamp column with column name `insertion_timestamp`.


In [1]:
# STEP 1
import polars as pl
df_cus = pl.read_csv('./data/northwind/customer.csv')
df_cus.shape

(30, 18)

In [2]:
# STEP 2
from datetime import datetime
def add_timestamp(df,colname):
    current_timestamp = datetime.now()
    df_n = df.with_columns(pl.lit(current_timestamp).alias(colname))
    # add เวลา เข้าไปใน df_cus
    # pl.lit เอาค่าไปพิมซ้ำๆ ในทุก record (ในกรณีนี้ใส่เวลาในทุก record)
    return df_n

def get_unigue(df,colname):
    return df.unique(colname)
# สร้างฟังชันก์ที่ใช้กำหนด primary key
# ถ้ามี id ซ้ำ มันจะลบตัวถัดไปออก

def Mysort(df,colname):
    return df.sort(colname)

def Rename_col(df,colnamedict):
    return df.rename(colnamedict)

def exclude_col(df,colname):
    return df.select(pl.col('*').exclude(colname))
# ลบคอลัมน์


In [3]:
# STEP 3
stg_customer = (df_cus
                .pipe(add_timestamp,'ingestion_timestamp')
                .pipe(get_unigue,'id')
                .pipe(Mysort,'id')
                .pipe(Rename_col,{'id':'customer_id'})
                )

In [4]:
stg_customer
# staging area เหมือนบ่อพัก ใช้ตรวจสอบก่อนนำเข้าไปใน ware house

customer_id,company,last_name,first_name,email_address,job_title,business_phone,home_phone,mobile_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page,notes,attachments,ingestion_timestamp
i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,datetime[μs]
1,"""Company A""","""Bedecs""","""Anna""",null,"""Owner""","""(123)555-0100""",null,null,"""(123)555-0101""","""123 1st Street""","""Seattle""","""WA""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.724453
2,"""Company B""","""Gratacos Solsona""","""Antonio""",null,"""Owner""","""(123)555-0100""",null,null,"""(123)555-0101""","""123 2nd Street""","""Boston""","""MA""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.724453
3,"""Company C""","""Axen""","""Thomas""",null,"""Purchasing Representative""","""(123)555-0100""",null,null,"""(123)555-0101""","""123 3rd Street""","""Los Angelas""","""CA""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.724453
4,"""Company D""","""Lee""","""Christina""",null,"""Purchasing Manager""","""(123)555-0100""",null,null,"""(123)555-0101""","""123 4th Street""","""New York""","""NY""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.724453
5,"""Company E""","""O’Donnell""","""Martin""",null,"""Owner""","""(123)555-0100""",null,null,"""(123)555-0101""","""123 5th Street""","""Minneapolis""","""MN""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.724453
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
25,"""Company Y""","""Rodman""","""John""",null,"""Purchasing Manager""","""(123)555-0100""",null,null,"""(123)555-0101""","""789 25th Street""","""Chicago""","""IL""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.724453
26,"""Company Z""","""Liu""","""Run""",null,"""Accounting Assistant""","""(123)555-0100""",null,null,"""(123)555-0101""","""789 26th Street""","""Miami""","""FL""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.724453
27,"""Company AA""","""Toh""","""Karen""",null,"""Purchasing Manager""","""(123)555-0100""",null,null,"""(123)555-0101""","""789 27th Street""","""Las Vegas""","""NV""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.724453


In [5]:
# load stg customer from stage repository
dim_customer = (stg_customer
                .pipe(exclude_col,'ingestion_timestamp')
                .pipe(add_timestamp,'insertion_timestamp')
                )
dim_customer

customer_id,company,last_name,first_name,email_address,job_title,business_phone,home_phone,mobile_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page,notes,attachments,insertion_timestamp
i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,datetime[μs]
1,"""Company A""","""Bedecs""","""Anna""",null,"""Owner""","""(123)555-0100""",null,null,"""(123)555-0101""","""123 1st Street""","""Seattle""","""WA""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.809768
2,"""Company B""","""Gratacos Solsona""","""Antonio""",null,"""Owner""","""(123)555-0100""",null,null,"""(123)555-0101""","""123 2nd Street""","""Boston""","""MA""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.809768
3,"""Company C""","""Axen""","""Thomas""",null,"""Purchasing Representative""","""(123)555-0100""",null,null,"""(123)555-0101""","""123 3rd Street""","""Los Angelas""","""CA""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.809768
4,"""Company D""","""Lee""","""Christina""",null,"""Purchasing Manager""","""(123)555-0100""",null,null,"""(123)555-0101""","""123 4th Street""","""New York""","""NY""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.809768
5,"""Company E""","""O’Donnell""","""Martin""",null,"""Owner""","""(123)555-0100""",null,null,"""(123)555-0101""","""123 5th Street""","""Minneapolis""","""MN""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.809768
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
25,"""Company Y""","""Rodman""","""John""",null,"""Purchasing Manager""","""(123)555-0100""",null,null,"""(123)555-0101""","""789 25th Street""","""Chicago""","""IL""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.809768
26,"""Company Z""","""Liu""","""Run""",null,"""Accounting Assistant""","""(123)555-0100""",null,null,"""(123)555-0101""","""789 26th Street""","""Miami""","""FL""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.809768
27,"""Company AA""","""Toh""","""Karen""",null,"""Purchasing Manager""","""(123)555-0100""",null,null,"""(123)555-0101""","""789 27th Street""","""Las Vegas""","""NV""",99999,"""USA""",null,null,null,2024-07-05 20:54:52.809768


---

## Employee dimension table.

1. Read employee table from `employees.csv` file.

2. Create timstamp column with column name `ingestion_timestamp` and name table as `stg_employee`, and load data into `stg_northwind` database. 

3. Transform data as follows:
    - Sort by `id` and get unique `id`.
    - Rename `id` columns to `employee_id`.
    - Delete an `ingestion_timestamp` column and add timstamp column with column name `insertion_timestamp`.


In [6]:
# STEP 1
import polars as pl
df_emp = pl.read_csv('./data/northwind/employees.csv')

In [7]:
# STEP 2
stg_employee = df_emp.pipe(add_timestamp,'ingestion_timestamp')
stg_employee

id,company,last_name,first_name,email_address,job_title,business_phone,home_phone,mobile_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page,notes,attachments,ingestion_timestamp
i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,datetime[μs]
2,"""Northwind Traders""","""Cencini""","""Andrew""","""andrew@northwindtraders.com""","""Vice President, Sales""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 2nd Avenue""","""Bellevue""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…","""Joined the company as a sales …",null,2024-07-05 20:54:52.850262
8,"""Northwind Traders""","""Giussani""","""Laura""","""laura@northwindtraders.com""","""Sales Coordinator""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 8th Avenue""","""Redmond""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…","""Reads and writes French.""",null,2024-07-05 20:54:52.850262
6,"""Northwind Traders""","""Neipper""","""Michael""","""michael@northwindtraders.com""","""Sales Representative""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 6th Avenue""","""Redmond""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…","""Fluent in Japanese and can rea…",null,2024-07-05 20:54:52.850262
3,"""Northwind Traders""","""Kotas""","""Jan""","""jan@northwindtraders.com""","""Sales Representative""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 3rd Avenue""","""Redmond""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…","""Was hired as a sales associate…",null,2024-07-05 20:54:52.850262
1,"""Northwind Traders""","""Freehafer""","""Nancy""","""nancy@northwindtraders.com""","""Sales Representative""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 1st Avenue""","""Seattle""","""WA""",99999,"""USA""","""#http://northwindtraders.com#""",null,null,2024-07-05 20:54:52.850262
7,"""Northwind Traders""","""Zare""","""Robert""","""robert@northwindtraders.com""","""Sales Representative""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 7th Avenue""","""Seattle""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…",null,null,2024-07-05 20:54:52.850262
4,"""Northwind Traders""","""Sergienko""","""Mariya""","""mariya@northwindtraders.com""","""Sales Representative""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 4th Avenue""","""Kirkland""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…",null,null,2024-07-05 20:54:52.850262
9,"""Northwind Traders""","""Hellung-Larsen""","""Anne""","""anne@northwindtraders.com""","""Sales Representative""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 9th Avenue""","""Seattle""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…","""Fluent in French and German.""",null,2024-07-05 20:54:52.850262
5,"""Northwind Traders""","""Thorpe""","""Steven""","""steven@northwindtraders.com""","""Sales Manager""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 5th Avenue""","""Seattle""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…","""Joined the company as a sales …",null,2024-07-05 20:54:52.850262


In [8]:
# STEP 3
stg_employee = (df_emp
                .pipe(get_unigue,'id')
                .pipe(Mysort,'id')
                .pipe(Rename_col,{'id':'employee_id'})
                .pipe(exclude_col,'ingestion_timestamp')
                .pipe(add_timestamp,'insertion_timestamp')
                )
stg_employee

employee_id,company,last_name,first_name,email_address,job_title,business_phone,home_phone,mobile_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page,notes,attachments,insertion_timestamp
i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,datetime[μs]
1,"""Northwind Traders""","""Freehafer""","""Nancy""","""nancy@northwindtraders.com""","""Sales Representative""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 1st Avenue""","""Seattle""","""WA""",99999,"""USA""","""#http://northwindtraders.com#""",null,null,2024-07-05 20:54:52.884531
2,"""Northwind Traders""","""Cencini""","""Andrew""","""andrew@northwindtraders.com""","""Vice President, Sales""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 2nd Avenue""","""Bellevue""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…","""Joined the company as a sales …",null,2024-07-05 20:54:52.884531
3,"""Northwind Traders""","""Kotas""","""Jan""","""jan@northwindtraders.com""","""Sales Representative""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 3rd Avenue""","""Redmond""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…","""Was hired as a sales associate…",null,2024-07-05 20:54:52.884531
4,"""Northwind Traders""","""Sergienko""","""Mariya""","""mariya@northwindtraders.com""","""Sales Representative""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 4th Avenue""","""Kirkland""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…",null,null,2024-07-05 20:54:52.884531
5,"""Northwind Traders""","""Thorpe""","""Steven""","""steven@northwindtraders.com""","""Sales Manager""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 5th Avenue""","""Seattle""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…","""Joined the company as a sales …",null,2024-07-05 20:54:52.884531
6,"""Northwind Traders""","""Neipper""","""Michael""","""michael@northwindtraders.com""","""Sales Representative""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 6th Avenue""","""Redmond""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…","""Fluent in Japanese and can rea…",null,2024-07-05 20:54:52.884531
7,"""Northwind Traders""","""Zare""","""Robert""","""robert@northwindtraders.com""","""Sales Representative""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 7th Avenue""","""Seattle""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…",null,null,2024-07-05 20:54:52.884531
8,"""Northwind Traders""","""Giussani""","""Laura""","""laura@northwindtraders.com""","""Sales Coordinator""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 8th Avenue""","""Redmond""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…","""Reads and writes French.""",null,2024-07-05 20:54:52.884531
9,"""Northwind Traders""","""Hellung-Larsen""","""Anne""","""anne@northwindtraders.com""","""Sales Representative""","""(123)555-0100""","""(123)555-0102""",null,"""(123)555-0103""","""123 9th Avenue""","""Seattle""","""WA""",99999,"""USA""","""http://northwindtraders.com#ht…","""Fluent in French and German.""",null,2024-07-05 20:54:52.884531


--- 

## production dimension table.

1. Read product table from `products.csv` file.

2. Transform data as follows:
    - Filter out records with `supplier_ids` containing ';'.
    - Change the column type of `supplier_ids` column to `pl.Int64`.
    - Rename `supplier_ids` columns to `supplier_id`.
    - Rename `id` columns to `product_id`.
    - Create timstamp column with column name `ingestion_timestamp` and name table as `stg_product`, and load data into `stg_northwind` database.  

3. Read suppliers table from `suppliers.csv` file. 

4. Create timstamp column with column name `ingestion_timestamp`, rename `id` to `supplier_id`, and name table as `stg_supplier`, and load data into `stg_northwind` database.

5. Transform data as follows:
    - Perform left join of `stg_product` table and the temporary table containing ['supplier_id','company'] obtained from `stg_supplier` table.
    - Select all columns exclude `supplier_id`.
    - get unique `product_id`.
    - Delete an `ingestion_timestamp` column and add timstamp column with column name `insertion_timestamp`.
    - Name the result table `dim_product`.

In [9]:
import polars as pl
df_product = pl.read_csv('./data/northwind/products.csv')

In [10]:
df_product.head(5)

supplier_ids,id,product_code,product_name,description,standard_cost,list_price,reorder_level,target_level,quantity_per_unit,discontinued,minimum_reorder_quantity,category,attachments
str,i64,str,str,str,f64,f64,i64,i64,str,i64,i64,str,str
"""2;6""",74,"""NWTDFN-74""","""Northwind Traders Almonds""",null,7.5,10.0,5,20,"""5 kg pkg.""",0,5,"""Dried Fruit & Nuts""",null
"""1""",19,"""NWTBGM-19""","""Northwind Traders Chocolate Bi…",null,6.9,9.2,5,20,"""10 boxes x 12 pieces""",0,5,"""Baked Goods & Mixes""",null
"""1""",21,"""NWTBGM-21""","""Northwind Traders Scones""",null,7.5,10.0,5,20,"""24 pkgs. x 4 pieces""",0,5,"""Baked Goods & Mixes""",null
"""6""",91,"""NWTCFV-91""","""Northwind Traders Cherry Pie F…",null,1.0,2.0,10,40,"""15.25 OZ""",0,null,"""Canned Fruit & Vegetables""",null
"""6""",90,"""NWTCFV-90""","""Northwind Traders Pineapple""",null,1.0,1.8,10,40,"""15.25 OZ""",0,null,"""Canned Fruit & Vegetables""",null


In [11]:
def filter_out(df,colname,patturn):
    return df.filter(~pl.col(colname).str.contains(patturn))

def change_dttype(df,colname,datatype):
    df = df.with_columns([
    pl.col(colname).cast(datatype)
])
    return df

In [12]:
stg_product = (df_product
    .pipe(filter_out,'supplier_ids',';')
    .pipe(change_dttype,'supplier_ids',pl.Int64)
    .pipe(Rename_col,{'supplier_ids':'supplier_id'})
    .pipe(Rename_col,{'id':'product_id'})
    .pipe(add_timestamp,'ingestion_timestamp')
)
stg_product

supplier_id,product_id,product_code,product_name,description,standard_cost,list_price,reorder_level,target_level,quantity_per_unit,discontinued,minimum_reorder_quantity,category,attachments,ingestion_timestamp
i64,i64,str,str,str,f64,f64,i64,i64,str,i64,i64,str,str,datetime[μs]
1,19,"""NWTBGM-19""","""Northwind Traders Chocolate Bi…",null,6.9,9.2,5,20,"""10 boxes x 12 pieces""",0,5,"""Baked Goods & Mixes""",null,2024-07-05 20:54:52.996930
1,21,"""NWTBGM-21""","""Northwind Traders Scones""",null,7.5,10.0,5,20,"""24 pkgs. x 4 pieces""",0,5,"""Baked Goods & Mixes""",null,2024-07-05 20:54:52.996930
6,91,"""NWTCFV-91""","""Northwind Traders Cherry Pie F…",null,1.0,2.0,10,40,"""15.25 OZ""",0,null,"""Canned Fruit & Vegetables""",null,2024-07-05 20:54:52.996930
6,90,"""NWTCFV-90""","""Northwind Traders Pineapple""",null,1.0,1.8,10,40,"""15.25 OZ""",0,null,"""Canned Fruit & Vegetables""",null,2024-07-05 20:54:52.996930
6,88,"""NWTCFV-88""","""Northwind Traders Pears""",null,1.0,1.3,10,40,"""15.25 OZ""",0,null,"""Canned Fruit & Vegetables""",null,2024-07-05 20:54:52.996930
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,97,"""NWTC-82""","""Northwind Traders Hot Cereal""",null,3.0,5.0,50,200,null,0,null,"""Cereal""",null,2024-07-05 20:54:52.996930
2,80,"""NWTDFN-80""","""Northwind Traders Dried Plums""",null,3.0,3.5,50,75,"""1 lb bag""",0,25,"""Dried Fruit & Nuts""",null,2024-07-05 20:54:52.996930
6,99,"""NWTSO-99""","""Northwind Traders Chicken Soup""",null,1.0,1.95,100,200,null,0,null,"""Soups""",null,2024-07-05 20:54:52.996930


In [13]:
import polars as pl
df_supplier = pl.read_csv('./data/northwind/suppliers.csv')

In [14]:
stg_supplier = (df_supplier
                .pipe(Rename_col,{'id':'supplier_id'})
                .pipe(add_timestamp,'ingestion_timestamp')
                )
stg_supplier

supplier_id,company,last_name,first_name,email_address,job_title,business_phone,home_phone,mobile_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page,notes,attachments,ingestion_timestamp
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,datetime[μs]
1,"""Supplier A""","""Andersen""","""Elizabeth A.""",null,"""Sales Manager""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697
2,"""Supplier B""","""Weiler""","""Cornelia""",null,"""Sales Manager""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697
5,"""Supplier E""","""Hernandez-Echevarria""","""Amaya""",null,"""Sales Manager""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697
9,"""Supplier I""","""Sandberg""","""Mikael""",null,"""Sales Manager""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697
10,"""Supplier J""","""Sousa""","""Luis""",null,"""Sales Manager""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697
4,"""Supplier D""","""Sato""","""Naoki""",null,"""Marketing Manager""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697
7,"""Supplier G""","""Glasson""","""Stuart""",null,"""Marketing Manager""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697
6,"""Supplier F""","""Hayakawa""","""Satomi""",null,"""Marketing Assistant""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697
3,"""Supplier C""","""Kelley""","""Madeleine""",null,"""Sales Representative""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697


In [41]:
product_supplier = stg_product.join(
    stg_supplier, on=['supplier_id','job_title'], how='left'
)

ColumnNotFoundError: job_title

In [32]:
dim_product = (product_supplier
                    .select(pl.col('*').exclude('supplier_id'))
                    .pipe(get_unigue,'product_id')
                    .pipe(exclude_col,'ingestion_timestamp')
                    .pipe(add_timestamp,'insertion_timestamp')
)
dim_product

product_id,product_code,product_name,description,standard_cost,list_price,reorder_level,target_level,quantity_per_unit,discontinued,minimum_reorder_quantity,category,attachments,company,last_name,first_name,email_address,job_title,business_phone,home_phone,mobile_phone,fax_number,address,city,state_province,zip_postal_code,country_region,web_page,notes,attachments_right,ingestion_timestamp_right,insertion_timestamp
i64,str,str,str,f64,f64,i64,i64,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,datetime[μs],datetime[μs]
5,"""NWTO-5""","""Northwind Traders Olive Oil""",null,16.0125,21.35,10,40,"""36 boxes""",0,10,"""Oil""",null,"""Supplier J""","""Sousa""","""Luis""",null,"""Sales Manager""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697,2024-07-05 21:21:38.473430
8,"""NWTS-8""","""Northwind Traders Curry Sauce""",null,30.0,40.0,10,40,"""12 - 12 oz jars""",0,10,"""Sauces""",null,"""Supplier H""","""Dunton""","""Bryn Paul""",null,"""Sales Representative""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697,2024-07-05 21:21:38.473430
91,"""NWTCFV-91""","""Northwind Traders Cherry Pie F…",null,1.0,2.0,10,40,"""15.25 OZ""",0,null,"""Canned Fruit & Vegetables""",null,"""Supplier F""","""Hayakawa""","""Satomi""",null,"""Marketing Assistant""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697,2024-07-05 21:21:38.473430
1,"""NWTB-1""","""Northwind Traders Chai""",null,13.5,18.0,10,40,"""10 boxes x 20 bags""",0,10,"""Beverages""",null,"""Supplier D""","""Sato""","""Naoki""",null,"""Marketing Manager""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697,2024-07-05 21:21:38.473430
4,"""NWTCO-4""","""Northwind Traders Cajun Season…",null,16.5,22.0,10,40,"""48 - 6 oz jars""",0,10,"""Condiments""",null,"""Supplier J""","""Sousa""","""Luis""",null,"""Sales Manager""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697,2024-07-05 21:21:38.473430
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
80,"""NWTDFN-80""","""Northwind Traders Dried Plums""",null,3.0,3.5,50,75,"""1 lb bag""",0,25,"""Dried Fruit & Nuts""",null,"""Supplier B""","""Weiler""","""Cornelia""",null,"""Sales Manager""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697,2024-07-05 21:21:38.473430
7,"""NWTDFN-7""","""Northwind Traders Dried Pears""",null,22.5,30.0,10,40,"""12 - 1 lb pkgs.""",0,10,"""Dried Fruit & Nuts""",null,"""Supplier B""","""Weiler""","""Cornelia""",null,"""Sales Manager""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697,2024-07-05 21:21:38.473430
97,"""NWTC-82""","""Northwind Traders Hot Cereal""",null,3.0,5.0,50,200,null,0,null,"""Cereal""",null,"""Supplier A""","""Andersen""","""Elizabeth A.""",null,"""Sales Manager""",null,null,null,null,null,null,null,null,null,null,null,null,2024-07-05 20:54:53.036697,2024-07-05 21:21:38.473430
